In [1]:
from astro.load import Loader
from astro.preprocess import Preprocessor, GroupedEventPreprocessor
from astro.transforms import GroupSplitter


from trace_minder.preprocess import TracePreprocessor
from trace_minder.align import GroupedAligner
from trace_minder.trace_aggregation import PrePostAggregator
from trace_minder.responders.rotated_responder import AUCDiff, AUCDiffResponders


import pandas as pd

import os
from pathlib import Path
from astro.config import Config

#### Paths

In [2]:
notebook_path = Path(os.getcwd())
root_path = notebook_path.parent.parent
env_path = root_path / ".env"

paths = Config.from_env(env_path)

#### Config

- Loading and preprocessing of traces and events
- 'Group' information to account for different mice having different block onsets
- Alignment configuration
- Post alignment processing, if any
- Configuration of the AUC calculater
- Configuration of the AUC bootstrap tester

In [3]:
loader = Loader(data_dir=paths.data_dir)

loader_preprocessor_first5 = Preprocessor(
    trace_preprocessor=TracePreprocessor(
        max_time=600,
        standardize=True,
        medfilt_kernel_size=None,
        resample_frequency=0.1,
        drop_na=True,
    ),
    grouped_event_preprocessor=GroupedEventPreprocessor(
        df_events_group_col="mouse_name",
        df_events_event_time_col="start_time",
        first_x_events=5,
    ),
)
loader.preprocessor = loader_preprocessor_first5


group_splitter = GroupSplitter(
    df_mice=loader.load_mice(),
    df_neurons=loader.load_neurons(),
    df_traces_time_col="time",
    excluded_groups=["VEH-VEH"],
    df_neurons_mouse_col="mouse_name",
    df_mice_mouse_col="mouse_name",
    df_neurons_neuron_col="cell_id",
    df_mice_group_col="group",
)

aligner = GroupedAligner(
    t_before=30,
    t_after=30,
    df_wide_group_mapper=group_splitter.neurons_by_mouse,
    df_events_event_time_col="start_time",
    df_events_group_col="mouse_name",
    round_precision=1,
)

average_trace_preprocessor = TracePreprocessor()

aggregator = PrePostAggregator(event_idx_col=None)


stat_calculator = AUCDiff(
    aligner=aligner,
    average_trace_preprocessor=average_trace_preprocessor,
    aggregator=aggregator,
)

responders_calculator = AUCDiffResponders(
    aligner=aligner,
    average_trace_preprocessor=average_trace_preprocessor,
    aggregator=aggregator,
    n_boot=50,
    _store_reps=True,
)

#### Examplar: Calculating the Observed Statistic


- Traces and events start times are loaded from a single session for each mouse
- The `AUCDiff` AUC calculater performs the following steps automatically:
  - Traces are aligned to events, calculating an average trace over trials
  - AUC is calculated for the pre and post sections of the average trace


In [4]:
df_traces = loader.load_traces(session_name="ret")
df_events = loader.load_blockstarts(session_name="ret", block_group="CS")


print("Average trace (long)")
df_average_trace = aligner.average_trace_long(df_traces, event_starts=df_events)
display(df_average_trace.head(3))


print()
print("Statistic: AUC-post minus AUC-pre")
df_auc_diff = aggregator.prepost_diff(df_average_trace)
display(df_auc_diff.head(3))


# stat calculater
print()
print("---------------------------")
print("Statistic calculator")
print("---------------------------")
stat_calculator.get_stat(df_traces, df_events).head(3)

Average trace (long)


,aligned_time,neuron,value
0,-30.0,1050,-0.141333
1,-29.9,1050,-0.014349
2,-29.8,1050,-0.063068



Statistic: AUC-post minus AUC-pre


,event_idx,neuron,post,pre,post_sub_pre
0,0,1050,0.015355,-0.079142,0.094497
1,0,1051,0.512359,0.215746,0.296614
2,0,1053,-0.105208,0.001938,-0.107147



---------------------------
Statistic calculator
---------------------------


,event_idx,cell_id,post,pre,auc_diff
0,0,1050,0.015355,-0.079142,0.094497
1,0,1051,0.512359,0.215746,0.296614
2,0,1053,-0.105208,0.001938,-0.107147


#### Exemplar: Computing responders using a bootstrap test

- Traces and events are loaded for each mice for a single session
- The `AUCDiffResponders` class calulates the following
  - Once, calculates the AUC difference statistic on the observed data
  - For each bootstrap iteration, rotates the traces, and calculates the AUC difference statistic
  - Calculates p-values for each cell by comparing the observed statistic to the bootstrap distribution

In [5]:
responders = responders_calculator.get_responders(df_traces, df_events)

/opt/miniconda3/envs/astro/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/miniconda3/envs/astro/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/miniconda3/envs/astro/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/miniconda3/envs/astro/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too sho

In [6]:
print("---------------------------")
print("DF OBSERVED")
print("---------------------------")
display(responders_calculator.df_obs_.head(3))

print("---------------------------")
print("DF BOOTSTRAP REPLICATES")
print("---------------------------")
display(responders_calculator.df_bootreps_.head(3))


print("---------------------------")
print("DF RESPONDERS")
print("---------------------------")
display(responders_calculator.df_responders_.head(3))

---------------------------
DF OBSERVED
---------------------------


,cell_id,stat
0,1050,0.094497
1,1051,0.296614
2,1053,-0.107147


---------------------------
DF BOOTSTRAP REPLICATES
---------------------------


,cell_id,stat,sample
0,1050,0.575329,0
0,1050,0.315194,1
0,1050,-0.299084,2


---------------------------
DF RESPONDERS
---------------------------


,cell_id,stat,pval,sig
0,1050,0.094497,1.68,False
1,1051,0.296614,1.12,False
2,1053,-0.107147,1.24,False
